<a href="https://colab.research.google.com/github/mwtam/blog/blob/main/Einstein's_riddle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import copy

In [56]:
class House:
    def __init__(self):
        self.person = None
        self.color = None
        self.pet = None
        self.drinks = None
        self.smoke = None

    def __str__(self):
        return f"{str(self.person):>10} {str(self.color):>8} {str(self.pet):>8} {str(self.drinks):>8} {str(self.smoke):>12}"


In [57]:
def print_answer(answer):
    for i, h in enumerate(answer, start=1):
        print(f"{i}: {h}")

def v(field, s):
    # v = valid: can fill the field with s
    # Either it is empty, or it is already the value s.
    return field is None or field == s

def vaild_answer(answer):
    person = set()
    color = set()
    pet = set()
    drinks = set()
    smoke = set()

    for h in answer:
        if h.person is not None:
            if h.person not in person:
                person.add(h.person)
            else:
                person.add(h.person)
                return False

        if h.color is not None:
            if h.color not in color:
                color.add(h.color)
            else:
                return False

        if h.pet is not None:
            if h.pet not in pet:
                pet.add(h.pet)
            else:
                return False

        if h.drinks is not None:
            if h.drinks not in drinks:
                drinks.add(h.drinks)
            else:
                return False

        if h.smoke is not None:
            if h.smoke not in smoke:
                smoke.add(h.smoke)
            else:
                return False

    return True


In [58]:
def norwegian_first(answers):
    for a in answers:
        if v(a[0].person, "Norwegian"):
            new_a = copy.deepcopy(a)
            new_a[0].person = "Norwegian"
            if vaild_answer(new_a):
                yield new_a

def green_left_to_white(answers):
    for a in answers:
        for i in range(len(a) - 1):
            if v(a[i].color, "Green") and v(a[i+1].color, "White"):
                new_a = copy.deepcopy(a)
                new_a[i].color = "Green"
                new_a[i+1].color = "White"
                if vaild_answer(new_a):
                    yield new_a

def mid_milk(answers):
    for a in answers:
        mid = len(a) // 2
        if v(a[mid].drinks, "Milk"):
            new_a = copy.deepcopy(a)
            new_a[mid].drinks = "Milk"
            if vaild_answer(new_a):
                yield new_a

def same_house(answers, relationship):
    f1, v1, f2, v2 = relationship.split()
    for a in answers:
        for i in range(len(a)):
            if v(getattr(a[i], f1), v1) and v(getattr(a[i], f2), v2):
                new_a = copy.deepcopy(a)
                setattr(new_a[i], f1, v1)
                setattr(new_a[i], f2, v2)
                if vaild_answer(new_a):
                    yield new_a

def next_house(answers, relationship):
    f1, v1, f2, v2 = relationship.split()
    for a in answers:
        for i in range(len(a) - 1):
            if v(getattr(a[i], f1), v1) and v(getattr(a[i+1], f2), v2):
                new_a = copy.deepcopy(a)
                setattr(new_a[i], f1, v1)
                setattr(new_a[i+1], f2, v2)
                if vaild_answer(new_a):
                    yield new_a
        for i in range(1, len(a)):
            if v(getattr(a[i], f1), v1) and v(getattr(a[i-1], f2), v2):
                new_a = copy.deepcopy(a)
                setattr(new_a[i], f1, v1)
                setattr(new_a[i-1], f2, v2)
                if vaild_answer(new_a):
                    yield new_a


In [59]:
def solve():
    initial_answer = [House() for _ in range(5)]

    g = [initial_answer]

    # Specific hints first
    g = norwegian_first(g)
    g = mid_milk(g)

    # This is Specific after Norwegian is fixed
    g = next_house(g, "person Norwegian color Blue")

    # Since Blue is know, add all color related rules
    g = same_house(g, "person Brit color Red")
    g = same_house(g, "smoke Dunhill color Yellow")
    g = same_house(g, "drinks Coffee color Green")
    g = green_left_to_white(g)

    # Then drinks, for there are two drinks appeared already
    g = next_house(g, "smoke Blend drinks Water")
    g = same_house(g, "smoke BlueMaster drinks Beer")
    g = same_house(g, "person Dane drinks Tea")

    # Finally smoke
    g = next_house(g, "smoke Dunhill pet Horse")
    g = same_house(g, "smoke Pallmall pet Bird")
    g = next_house(g, "smoke Blend pet Cat")
    g = same_house(g, "person German smoke Prince")

    # The final hint
    g = same_house(g, "person Swede pet Dog")

    for i, a in enumerate(g):
        # The hints never mention fish.
        # It is the final None in the answer.
        for h in a:
            if h.pet is None:
                h.pet = "* FISH *"

        print("=" * 10, i, "=" * 10)
        print_answer(a)
        # ========== 0 ==========
        # 1:  Norwegian   Yellow      Cat    Water      Dunhill
        # 2:       Dane     Blue    Horse      Tea        Blend
        # 3:       Brit      Red     Bird     Milk     Pallmall
        # 4:     German    Green * FISH *   Coffee       Prince
        # 5:      Swede    White      Dog     Beer   BlueMaster


In [60]:
solve()

========== 0 ==========
1:  Norwegian   Yellow      Cat    Water      Dunhill
2:       Dane     Blue    Horse      Tea        Blend
3:       Brit      Red     Bird     Milk     Pallmall
4:     German    Green * FISH *   Coffee       Prince
5:      Swede    White      Dog     Beer   BlueMaster
